In [4]:
import os
os.chdir("/Desktop/Academic/")

import math
import easyplotlib as epl
from easyplotlib import *

def list_coord(lis, t):
    """
    Make a function that takes a list of points in the plane and a number t (that can be either
    0 or 1) and uses list comprehension to return the list of the t-coordinates of the points.
    """

    return [point[t] for point in lis]

def list_by_two_lists_insertion_sort(orglis, complisone, complistwo):
    """
   takes any type of list and makes a new list that is the result of
   sorting the original list by a list of numbers, and secondarily by a second list of numbers in case
   of a tie in the first list of numbers, using Insertion Sort. The original list should not be changed.
    """
    # make copies of the input lists to keep the original lists unchanged
    newlis = orglis[:]
    primary = complisone[:]
    secondary = complistwo[:]

    # apply Insertion Sort based on the primary and secondary lists
    n = len(newlis)
    for i in range(1, n):
        # save the current elements to be compared and inserted
        current_item = newlis[i]
        current_primary = primary[i]
        current_secondary = secondary[i]

        j = i - 1
        # use the primary list for ordering, and secondary list for tie-breaking
        while j >= 0 and (primary[j] > current_primary or
                          (primary[j] == current_primary and secondary[j] > current_secondary)):
            newlis[j + 1] = newlis[j]
            primary[j + 1] = primary[j]
            secondary[j + 1] = secondary[j]
            j -= 1

        # place the current element at its sorted position
        newlis[j + 1] = current_item
        primary[j + 1] = current_primary
        secondary[j + 1] = current_secondary

    return newlis

def list_bottom_left(points):
    """
    Finds the bottom-left point in a list of points and returns it along with a list of sorted remaining points.
    """
    # Extract y-coordinates (primary) and x-coordinates (secondary) from the points
    y_coords = list_coord(points, 1)
    x_coords = list_coord(points, 0)

    # Sort the points using insertion sort
    sorted_points = list_by_two_lists_insertion_sort(points, y_coords, x_coords)

    # Identify the bottom-left point and remaining points
    bottom_left = sorted_points[0]
    remaining_points = sorted_points[1:]

    return [bottom_left, remaining_points]

def cos_polar_angle(p1, p2):
    """
    Calculates the negative cosine of the polar angle between the vector
    formed by p1 and p2 and the positive x-axis.
    Returns: float: The negative cosine of the polar angle.
    """
    # Convert points to vectors
    vector = Matrix([p2[0] - p1[0], p2[1] - p1[1]])
    unit_vector = Matrix([1, 0])  # Unit vector in the direction of the positive x-axis

    # Compute the dot product and magnitudes
    dot_product = vector.dot(unit_vector)
    vector_magnitude = vector.norm()

    # Compute the cosine of the polar angle
    cos_theta = dot_product / vector_magnitude

    return -cos_theta

def distance(p1, p2):
    """
    Calculate the Euclidean distance between two points.
    """
    dx = p2[0] - p1[0]
    dy = p2[1] - p1[1]
    return math.sqrt(dx**2 + dy**2)

def list_polar_angle_sort(points):
    """
    Ranks points in the plane by increasing polar angle with the bottom-left point,
    and secondarily by increasing distance from the bottom-left point.
    Uses the list_by_two_lists_insertion_sort for sorting.
    Returns:list: The bottom-left point followed by the ranked points.
    """
    # Step 1: Find the bottom-left point
    bottom_left, remaining_points = list_bottom_left(points)

    # Step 2: Calculate the negative cosine of polar angles and distances using list comprehension
    neg_cosines = [cos_polar_angle(bottom_left, p) for p in remaining_points]
    distances = [distance(bottom_left, p) for p in remaining_points]

    # Step 3: Sort the remaining points using list_by_two_lists_insertion_sort
    sorted_points = list_by_two_lists_insertion_sort(remaining_points, neg_cosines, distances)

    # Step 4: Return the bottom-left point and the sorted list
    return [bottom_left, sorted_points]


FileNotFoundError: [Errno 2] No such file or directory: '/Desktop/Academic/'

In [ ]:
from ipywidgets import interact, widgets, Layout
import ast

# If these are in your project, import them; example:
# from your_module import list_polar_angle_sort, points_relative_plot, line_segment_plot, label_plot, eplot
import easyplotlib as epl
from easyplotlib import eplot, points_relative_plot, line_segment_plot, label_plot  # adjust if needed

def parse_points(s: str):
    """Safe parser for a list of (x, y) tuples typed into the Text widget."""
    try:
        pts = ast.literal_eval(s)
        if not isinstance(pts, (list, tuple)):
            raise ValueError("Parsed value is not a list/tuple.")
        return [tuple(map(float, p)) for p in pts]
    except Exception as e:
        raise ValueError(
            "Couldn't parse points. Use format like [(x1, y1), (x2, y2)]. "
            f"Error: {e}"
        )

@interact(
    points=widgets.Text(
        value="[(10, 6), (8, 2), (2, 6), (10, 8), (4, 11), (7, 8), (7, 5), (4, 2), (5, 8), (3, 6)]",
        description="Points:",
        layout=Layout(width='90%')
    ),
    xrange=widgets.IntRangeSlider(value=(0, 20), min=-20, max=30, step=1, description="X-Range"),
    yrange=widgets.IntRangeSlider(value=(0, 20), min=-20, max=30, step=1, description="Y-Range"),
)
def interactive_plot(points, xrange, yrange):
    """
    Plots the points and the order in which they are sorted.
    """
    pts = parse_points(points)

    # Your function that returns (bottom_left, sorted_points)
    bottom_left, sorted_points = list_polar_angle_sort(pts)

    # Draw points
    points_plot = points_relative_plot(pts, xrange, yrange, face_color='blue', radius=0.2, fill=True)

    # Highlight the bottom-left point
    bottom_left_plot = points_relative_plot([bottom_left], xrange, yrange, face_color='red', radius=0.3, fill=True)

    # Lines from bottom-left to each sorted point
    lines = []
    for p in sorted_points:
        # line_segment_plot returns a list; extend rather than append
        lines += line_segment_plot([bottom_left, p], edge_color='green', line_width=1)

    # Labels for each point
    labels = []
    for p in pts:
        labels += label_plot(str(p), p, angle=0, distance=0.3, label_font={'font_size': 10})

    # Combine & render
    eplot(points_plot + bottom_left_plot + lines + labels, xrange, yrange)

    # Printed results
    print("Bottom Left Point:", bottom_left)
    print("Sorted Points by Polar Angle and Distance:", sorted_points)
